# Import

In [ ]:
import sympy as sp
import numpy as np
import itertools
from sympy import symbols, Eq, solve, diff, Function, simplify
from IPython.display import display, Math

# Calibration


In [ ]:
"""
f is the direct kinematics (written as a symbolic matrix)
r is the respective task measured coordinate (valued)
q is a list of configurations
phi is a symbolic matrix multiplying the variations (see notes)
"""

def calibration_step(r, f, q, phi):

  print("\nStarting f (direct kinematics):\n")
  for i in range(len(f)):
    print(f[i])

  print("\nTask r:\n", r)

  l = len(q)
  print(f"\nConfigurations tried ({l}): \n")

  delta_r = []
  m,n = phi.shape
  phi_bar = np.zeros((m*l, n))

  k = 0
  for i in range(l):
    print(f"\nEXPERIMENT N.{i}: ", q[i])
    f_c = f.copy()
    phi_c = phi.copy()
    for n in range(len(q[i])):
      q_i = sp.symbols('q'+str(n+1))
      f_c = f_c.subs(q_i, q[i][n])
      phi_c = phi_c.subs(q_i, q[i][n])
    r_nom = list(f_c)
    print("\nr for this experiment: ", r[i])
    print("\nr_nom for this experiment: ", r_nom)
    print("\nphi for this experiment: ", phi_c)

    for j in range(len(r[i])):

      diff = r[i][j]-r_nom[j]
      if abs(diff) < 1e-15:
        diff = 0
      delta_r.append(diff)

    for row in range(phi_c.shape[0]):
      for col in range(phi_c.shape[1]):
        if abs(phi_c[row,col])<1e-10:
          phi_bar[k+row][col] = 0
        else:
          phi_bar[k+row][col] = phi_c[row,col]
    k += m

  delta_r = np.array(delta_r).reshape(-1,1)
  print("\ndelta r:\n", delta_r)
  print("\nphi bar:\n", phi_bar)

  # solution
  displacements = np.dot(np.linalg.pinv(phi_bar), delta_r)
  print("\nDisplacements:\n", displacements)

  return list(displacements)


In [ ]:
# example 2R robot midterm2022
l1 = sp.symbols('l1')
l2 = sp.symbols('l2')
c1 = sp.cos('q1')
s1 = sp.sin('q1')
c2 = sp.cos('q2')
s2 = sp.sin('q2')
c12 = sp.cos('q1+q2')
s12 = sp.sin('q1+q2')

l1_nom = 1
l2_nom = 1
f = sp.Matrix([ [l1_nom*c1 + l2_nom*c12],
                [l1_nom*s1 + l2_nom*s12]
              ])

r = [[2,0], [0,2], [1.6925, 0.7425], [1.7218, 0.6718]]
q = [(0,0), (np.pi/2,0), (np.pi/4, -np.pi/4), (0, np.pi/4)]

phi = sp.Matrix([
                  [c1, c12],
                  [s1, s12]
                ])

displ = calibration_step(r,f,q, phi)
print(f"\nAfter one iteration:\nL1 = {l1_nom-displ[0]}\nL2 = {l2_nom-displ[1]}")


Starting f (direct kinematics):

cos(q1) + cos(q1 + q2)
sin(q1) + sin(q1 + q2)

Task r:
 [[2, 0], [0, 2], [1.6925, 0.7425], [1.7218, 0.6718]]

Configurations tried (4): 


EXPERIMENT N.0:  (0, 0)

r for this experiment:  [2, 0]

r_nom for this experiment:  [2, 0]

phi for this experiment:  Matrix([[1, 1], [0, 0]])

EXPERIMENT N.1:  (1.5707963267948966, 0)

r for this experiment:  [0, 2]

r_nom for this experiment:  [1.22464679914735e-16, 2.00000000000000]

phi for this experiment:  Matrix([[6.12323399573677e-17, 6.12323399573677e-17], [1.00000000000000, 1.00000000000000]])

EXPERIMENT N.2:  (0.7853981633974483, -0.7853981633974483)

r for this experiment:  [1.6925, 0.7425]

r_nom for this experiment:  [1.70710678118655, 0.707106781186547]

phi for this experiment:  Matrix([[0.707106781186548, 1], [0.707106781186547, 0]])

EXPERIMENT N.3:  (0, 0.7853981633974483)

r for this experiment:  [1.7218, 0.6718]

r_nom for this experiment:  [1.70710678118655, 0.707106781186547]

phi for this e

# Redundancy: velocity level

## Jacobian-based methods

In [ ]:
def DLS(J, r_dot, values, mu):
  print("\nStarting Jacobian:\n", J)
  print("\nDesired task velocity:\n", r_dot)
  m,n = J.shape

  check_J = input("\nIs J symbolic? (Y/N): ")
  if check_J == 'Y':
    J = np.round(np.array(J.subs(values)).astype(np.float64), 3)

  I = np.identity(m)
  JJT = np.dot(J, J.T)
  mu2I = mu**2 * I
  inverse_part = np.linalg.inv(JJT + mu2I)
  q_dot = np.dot(np.dot(J.T, inverse_part), r_dot)
  print("\nThe solution q_dot is:\n", q_dot)
  return q_dot


In [ ]:
# example DLS

l = sp.symbols('l')
s1 = sp.sin('q1')
s2 = sp.sin('q2')
s3 = sp.sin('q3')
s12 = sp.sin('q1+q2')
s123 = sp.sin('q1+q2+q3')
c1 = sp.cos('q1')
c2 = sp.cos('q2')
c3 = sp.cos('q3')
c12 = sp.cos('q1+q2')
c123 = sp.cos('q1+q2+q3')

J = sp.Matrix([
    [-l*(s1+s12+s123), -l*(s12+s123), -l*s123],
    [l*(c1+c12+c123), l*(c12+c123), l*c123]
])

values = {'l':1, 'q1':np.pi/2, 'q2':np.pi/3, 'q3':-2*np.pi/3}

r_dot = np.array([[1], [-np.sqrt(3)]])
q_dot = DLS(J, r_dot, values, mu = 0.25)


Starting Jacobian:
 Matrix([[-l*(sin(q1) + sin(q1 + q2) + sin(q1 + q2 + q3)), -l*(sin(q1 + q2) + sin(q1 + q2 + q3)), -l*sin(q1 + q2 + q3)], [l*(cos(q1) + cos(q1 + q2) + cos(q1 + q2 + q3)), l*(cos(q1 + q2) + cos(q1 + q2 + q3)), l*cos(q1 + q2 + q3)]])

Desired task velocity:
 [[ 1.        ]
 [-1.73205081]]

Is J symbolic? (Y/N): Y

The solution q_dot is:
 [[-0.03026534]
 [-0.01513267]
 [-1.84678173]]


In [ ]:
# strong-right inverse
"""
- J is either symbolic or numeric Jacobian matrix
- r_dot is a vector of task velocity
- values is a dictionary with keys the symbolic variables names and as values
  the values to give to those variables
"""
def strong_right_inverse(J, r_dot, values):
  print("\nStarting Jacobian:\n", J)
  print("\nDesired task velocity:\n", r_dot)
  m,n = J.shape
  print(f"\nRows = {m}\nCols = {n}")

  non_singular_minors = []
  largest_size = 0

  check_J = input("\nIs J symbolic? (Y/N): ")
  if check_J == 'Y':
    J = np.round(np.array(J.subs(values)).astype(np.float64), 3)


  for col_comb in itertools.combinations(range(n), m):
    minor = J[:, col_comb]

    if np.linalg.det(minor) != 0:
      non_singular_minors.append(minor)
      largest_size = m

  l = len(non_singular_minors)
  if l > 0:
    print(f"\nFound {l} non-singular minors of J...\n")
    for i in range(l):
      print(non_singular_minors[i])
      print()

    K = np.zeros((n,m))
    J_a = non_singular_minors[0]  # change the index if you want to use another minor
    for i in range(m):
      for j in range(m):
        K[i][j] = J_a[i][j]

    print("\nThe strong right inverse using the first minor is:\n",K)
    q_dot = np.dot(K, r_dot)
    print("\nThe solution q_dot is:\n", q_dot)
  else:
    print("\nNo minor is non singular")

  return K, q_dot

In [ ]:
# example SRI
J = np.array([[1, 2, 3, 4],
              [5, 6, 7, 8],
              [9, 7, 11, 12]])
r_dot = np.array([
                  [1],
                  [0],
                  [-2]])
values = {'q1': 0, 'q2': 0, 'q3': 0, 'q4': 0}
K, q_dot = strong_right_inverse(J, r_dot, values)


Starting Jacobian:
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9  7 11 12]]

Desired task velocity:
 [[ 1]
 [ 0]
 [-2]]

Rows = 3
Cols = 4

Is J symbolic? (Y/N): N

Found 3 non-singular minors of J...

[[ 1  2  3]
 [ 5  6  7]
 [ 9  7 11]]

[[ 1  2  4]
 [ 5  6  8]
 [ 9  7 12]]

[[ 2  3  4]
 [ 6  7  8]
 [ 7 11 12]]


The strong right inverse using the first minor is:
 [[ 1.  2.  3.]
 [ 5.  6.  7.]
 [ 9.  7. 11.]
 [ 0.  0.  0.]]

The solution q_dot is:
 [[ -5.]
 [ -9.]
 [-13.]
 [  0.]]


In [ ]:
# pseudoinverse
"""
- J is either symbolic or numeric Jacobian matrix
- r_dot is a vector of task velocity
- values is a dictionary with keys the symbolic variables names and as values
  the values to give to those variables
"""

def pseudoinverse(J, r_dot, values):
  print("\nStarting jacobian:\n", J)
  print("\nDesired task velocity:\n", r_dot)

  check_J = input("\nIs J symbolic? (Y/N): ")
  if check_J == 'Y':
    J = np.round(np.array(J.subs(values)).astype(np.float64), 3)

  m,n = J.shape

  non_singular_minors = []
  for col_comb in itertools.combinations(range(n), m):
    minor = J[:, col_comb]

    if np.linalg.det(minor) != 0:
      non_singular_minors.append(minor)

  if len(non_singular_minors) > 0:
    print("\nJ is full row rank...\nLet's compute J#\n")
    J_pseudoinv = np.dot(J.T, np.linalg.inv(np.dot(J,J.T)))
  else:
    print("\nJ is not full row rank...\nLet's compute J#\n")
    J_pseudoinv = np.linalg.pinv(J)

  print("\nThe jacobian pseudoinverse is:\n", J_pseudoinv)
  q_dot = np.dot(J_pseudoinv, r_dot)
  print("\nThe solution q_dot is:\n", q_dot)
  return J_pseudoinv, q_dot

In [ ]:
# jacobian instanciation

# example DLS

s1 = sp.sin('q1')
s2 = sp.sin('q2')
s3 = sp.sin('q3')
s4 = sp.sin('q4')
s12 = sp.sin('q1+q2')
s123 = sp.sin('q1+q2+q3')
s1234 = sp.sin('q1+q2+q3+q4')
c1 = sp.cos('q1')
c2 = sp.cos('q2')
c3 = sp.cos('q3')
c4 = sp.cos('q4')
c12 = sp.cos('q1+q2')
c123 = sp.cos('q1+q2+q3')
c1234 = sp.cos('q1+q2+q3+q4')

J1 = sp.Matrix([
    [-(s1+s12+s123+s1234), -(s12+s123+s1234), -s123-s1234, -s1234],
    [(c1+c12+c123+c1234), (c12+c123+c1234), c123+c1234, c1234]
])

J2 = sp.Matrix([
    [-(s1+s12), -s12, 0, 0],
    [c1+c12, c12, 0, 0]
])

values = {'q1':0, 'q2':np.pi/6, 'q3':-np.pi/3, 'q4':-np.pi/3}

print(J2.subs(values))
#J = J.subs(values)

r_dot = np.array([-0.5, 0.866])

P,q = pseudoinverse(J2, r_dot, values)

#r_dot = np.array([[1], [-np.sqrt(3)]])


Matrix([[-0.500000000000000, -0.500000000000000, 0, 0], [1.86602540378444, 0.866025403784439, 0, 0]])

Starting jacobian:
 Matrix([[-sin(q1) - sin(q1 + q2), -sin(q1 + q2), 0, 0], [cos(q1) + cos(q1 + q2), cos(q1 + q2), 0, 0]])

Desired task velocity:
 [-0.5    0.866]

Is J symbolic? (Y/N): Y

J is full row rank...
Let's compute J#


The jacobian pseudoinverse is:
 [[ 1.732  1.   ]
 [-3.732 -1.   ]
 [ 0.     0.   ]
 [ 0.     0.   ]]

The solution q_dot is:
 [1.11022302e-16 1.00000000e+00 0.00000000e+00 0.00000000e+00]


In [ ]:
error = r_dot - np.dot(np.array(J.subs(values)),q)
print(error)
#print(np.linalg.norm(error))

[-2.22044604925031e-16 1.24013327302830e-5]


In [ ]:
# example pseudoinverse
J = np.array([[1, 1, 3, 4],
              [5, 2, 7, 8],
              [9, 1, 11, 12]])
r_dot = np.array([
                  [1],
                  [0],
                  [-2]])

values = {'q1': 0, 'q2': 0, 'q3': 0, 'q4': 0}
J_p, q_dot = pseudoinverse(J, r_dot, values)


Starting jacobian:
 [[ 1  1  3  4]
 [ 5  2  7  8]
 [ 9  1 11 12]]

Desired task velocity:
 [[ 1]
 [ 0]
 [-2]]

Is J symbolic? (Y/N): N

J is full row rank...
Let's compute J#


The jacobian pseudoinverse is:
 [[-0.69642857  0.35714286 -0.01785714]
 [-0.5         1.         -0.5       ]
 [ 0.08928571 -0.07142857  0.05357143]
 [ 0.48214286 -0.28571429  0.08928571]]

The solution q_dot is:
 [[-0.66071429]
 [ 0.5       ]
 [-0.01785714]
 [ 0.30357143]]


In [ ]:
# weighted pseudoinverse
"""
- J is either symbolic or numeric Jacobian matrix
- W is a (diagonal) weight matrix
- r_dot is a vector of task velocity
- values is a dictionary with keys the symbolic variables names and as values
  the values to give to those variables
"""
def weighted_pseudoinverse(J, W, r_dot, values):
  print("\nStarting jacobian:\n", J)
  print("\nWeight matrix:\n", W)
  print("\nDesired task velocity:\n", r_dot)

  check_J = input("\nIs J symbolic? (Y/N): ")
  if check_J == 'Y':
    J = np.round(np.array(J.subs(values)).astype(np.float64), 3)

  m,n = J.shape
  non_singular_minors = []
  for col_comb in itertools.combinations(range(n), m):
    minor = J[:, col_comb]

    if np.linalg.det(minor) != 0:
      non_singular_minors.append(minor)

  if len(non_singular_minors) > 0:
    print("\nJ is full row rank...\nLet's compute J#\n")
    w_inv = np.linalg.inv(W)
    last_term = np.linalg.inv(np.dot(np.dot(J, w_inv), J.T))
    J_pseudo_w = np.dot(np.dot(w_inv, J.T), last_term)
    print("\nThe weighted pseudoinverse is:\n", J_pseudo_w)
    q_dot = np.dot(J_pseudo_w, r_dot)
    print("\nThe solution q_dot is:\n", q_dot)
    return J_pseudo_w, q_dot
  else:
    print("\nJ is not full row rank... Can not compute the solution")
    return

In [ ]:
# example weighted pseudoinverse

J = np.array([[1, 0, 3, 4],
              [5, 1, 7, 8],
              [9, 0, 1, 12]])
W = np.array([
              [1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]
            ])
r_dot = np.array([
                  [1],
                  [0],
                  [-2]])

values = {'q1': 0, 'q2': 0, 'q3': 0, 'q4': 0}
J_w, q_dot = weighted_pseudoinverse(J, W, r_dot, values)


Starting jacobian:
 [[ 1  0  3  4]
 [ 5  1  7  8]
 [ 9  0  1 12]]

Weight matrix:
 [[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]

Desired task velocity:
 [[ 1]
 [ 0]
 [-2]]

Is J symbolic? (Y/N): N

J is full row rank...
Let's compute J#


The weighted pseudoinverse is:
 [[-0.5657232   0.23027105  0.03921804]
 [-0.25353802  0.13648357 -0.02206764]
 [-0.0492924   0.17270329 -0.09558647]
 [ 0.4284001  -0.18709523  0.06188534]]

The solution q_dot is:
 [[-0.64415927]
 [-0.20940273]
 [ 0.14188055]
 [ 0.30462941]]


## Null Space methods

In [ ]:
def chooce_q0_dot(obj, parameters):
  if obj == 'M':      # manipulability
    pass
  elif obj == 'JR':   # joint range
    pass
  elif obj == 'OA':   # obstacle avoidance
    pass

  return

In [ ]:
"""
- J is either symbolic or numeric Jacobian matrix
- r_dot is a vector of task velocity
- q_dot_zero is the preferred joint velocity
- values is a dictionary with keys the symbolic variables names and as values
  the values to give to those variables
"""

def projected_gradient(J, r_dot, q_dot_zero, values):
  print("\nStarting Jacobian:\n", J)
  print("\nDesired velocity:\n", r_dot)
  print("\nPreferred joint velocity:\n", q_dot_zero)
  check_J = input("\nIs J symbolic? (Y/N): ")
  if check_J == 'Y':
    J = np.round(np.array(J.subs(values)).astype(np.float64), 3)

  m,n = J.shape
  I = np.identity(n)
  J_pseudoinv = np.linalg.pinv(J)
  q_dot = np.dot(J_pseudoinv, r_dot) + np.dot(I-np.dot(J_pseudoinv, J), q_dot_zero)
  print("\nThe solution of PG method q_dot is:\n", q_dot)
  return q_dot

In [ ]:
# example PG

l = sp.symbols('l')
s1 = sp.sin('q1')
s2 = sp.sin('q2')
s3 = sp.sin('q3')
s12 = sp.sin('q1+q2')
s123 = sp.sin('q1+q2+q3')
c1 = sp.cos('q1')
c2 = sp.cos('q2')
c3 = sp.cos('q3')
c12 = sp.cos('q1+q2')
c123 = sp.cos('q1+q2+q3')

J = sp.Matrix([
    [-l*(s1+s12+s123), -l*(s12+s123), -l*s123],
    [l*(c1+c12+c123), l*(c12+c123), l*c123]
])

values = {'l':1, 'q1':np.pi/2, 'q2':np.pi/3, 'q3':-2*np.pi/3}

r_dot = np.array([[1], [-np.sqrt(3)]])
q_dot_zero = np.array([[-1], [0], [1]])
q_dot = projected_gradient(J, r_dot, q_dot_zero, values)


Starting Jacobian:
 Matrix([[-l*(sin(q1) + sin(q1 + q2) + sin(q1 + q2 + q3)), -l*(sin(q1 + q2) + sin(q1 + q2 + q3)), -l*sin(q1 + q2 + q3)], [l*(cos(q1) + cos(q1 + q2) + cos(q1 + q2 + q3)), l*(cos(q1 + q2) + cos(q1 + q2 + q3)), l*cos(q1 + q2 + q3)]])

Desired velocity:
 [[ 1.        ]
 [-1.73205081]]

Preferred joint velocity:
 [[-1]
 [ 0]
 [ 1]]

Is J symbolic? (Y/N): Y

The solution of PG method q_dot is:
 [[-0.19998827]
 [ 0.40000587]
 [-2.00005867]]


In [ ]:
"""
- J is either symbolic or numeric matrix jacobian
- H is either symbolic or numeric matrix defining the objective function
- r_dot is the vector representing the task velocity
- values is a dictionary with keys the variables and values the values to give
  to those variables
- variables is a n-ple containing variables defined as symbols
"""

def reduced_gradient(J, H, r_dot, values, variables):
  print("\nStarting J:\n", J)
  print("\nH:\n", H)
  print("\nValues:\n", values)

  check_J = input("\nIs J symbolic? (Y/N): ")
  if check_J == 'Y':
    J = np.round(np.array(J.subs(values)).astype(np.float64), 3)
  m,n = J.shape
  non_singular_minors = {}
  for col_comb in itertools.combinations(range(n), m):
    minor = J[:, col_comb]
    det = np.linalg.det(minor)
    if det != 0:
      non_singular_minors[det] = [minor, col_comb]

  selected_key = max(non_singular_minors, key=abs) # we choose the maximum determinant
  selected_minor = non_singular_minors[selected_key][0]
  selected_q = non_singular_minors[selected_key][1]

  q_a = []
  a = []
  for i in range(len(selected_q)):
    q_a.append(f"q{selected_q[i]+1}")
    a.append(selected_q[i])

  q_b = []
  b = []
  for i in range(1, n):
    if 'q'+str(i) not in q_a:
      q_b.append(f"q"+str(i))
      b.append(i-1)

  idx = a+b
  print(idx)
  T = np.zeros((n,n))
  for i in range(n):
    T[i][idx[i]] = 1

  print("\nT:\n", T)
  print(f"Choosen q_a: {q_a}\nRemaining q_b: {q_b}")

  J_a = selected_minor
  print("\nJ_a is:\n", J_a)
  J_a_inv = np.linalg.inv(J_a)

  J_b = J[:, b]
  print("\nJ_b is:\n", J_b)

  check_H = input("\nIs H symbolic? (Y/N): ")
  if check_H == 'Y':
    delta_H = sp.Matrix([sp.diff(H, var) for var in variables])
    print("\nGradient of H:\n", delta_H)
    v_DH = np.round(np.array(delta_H.subs(values)).astype(np.float64), 3)
    print("\nValued gradient of H:\n", v_DH)
  else:
    v_DH = H

  I = np.identity(n-m)
  DH_prime = np.dot(np.hstack((-np.dot(J_a_inv, J_b).T, I)), v_DH)
  print("\nThe reduced gradient is:\n", DH_prime)

  M1 = np.dot(J_a_inv, r_dot - np.dot(J_b, DH_prime))
  M = np.vstack((M1, DH_prime))
  q_dot = np.dot(T.T, M)
  print("\nThe solution q_dot is:\n", q_dot)
  return q_dot

In [ ]:
# example RG
# symbolic version midterm 29-03-2017

l = sp.symbols('l')
s1 = sp.sin('q1')
s2 = sp.sin('q2')
s3 = sp.sin('q3')
s12 = sp.sin('q1+q2')
s123 = sp.sin('q1+q2+q3')
c1 = sp.cos('q1')
c2 = sp.cos('q2')
c3 = sp.cos('q3')
c12 = sp.cos('q1+q2')
c123 = sp.cos('q1+q2+q3')

J = sp.Matrix([
    [-l*(s1+s12+s123), -l*(s12+s123), -l*s123],
    [l*(c1+c12+c123), l*(c12+c123), l*c123]
])

values = {'l':1, 'q1':np.pi/2, 'q2':np.pi/3, 'q3':-2*np.pi/3}
q1 = sp.symbols('q1')
q2 = sp.symbols('q2')
q3 = sp.symbols('q3')
variables = (q1, q2, q3)
H = s2**2 + s3**2
r_dot = np.array([[1], [-np.sqrt(3)]])
q_dot = reduced_gradient(J, H, r_dot, values, variables)


Starting J:
 Matrix([[-l*(sin(q1) + sin(q1 + q2) + sin(q1 + q2 + q3)), -l*(sin(q1 + q2) + sin(q1 + q2 + q3)), -l*sin(q1 + q2 + q3)], [l*(cos(q1) + cos(q1 + q2) + cos(q1 + q2 + q3)), l*(cos(q1 + q2) + cos(q1 + q2 + q3)), l*cos(q1 + q2 + q3)]])

H:
 sin(q2)**2 + sin(q3)**2

Values:
 {'l': 1, 'q1': 1.5707963267948966, 'q2': 1.0471975511965976, 'q3': -2.0943951023931953}

Is J symbolic? (Y/N): Y
[0, 2, 1]

T:
 [[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
Choosen q_a: ['q1', 'q3']
Remaining q_b: ['q2']

J_a is:
 [[-2.    -0.5  ]
 [ 0.     0.866]]

J_b is:
 [[-1.]
 [ 0.]]

Is H symbolic? (Y/N): Y

Gradient of H:
 Matrix([[0], [2*sin(q2)*cos(q2)], [2*sin(q3)*cos(q3)]])

Valued gradient of H:
 [[0.   ]
 [0.866]
 [0.866]]

The reduced gradient is:
 [[0.866]]

The solution q_dot is:
 [[-0.43298533]
 [ 0.866     ]
 [-2.00005867]]


## Task Augmentation

In [ ]:
"""
- J is the starting jacobian
- l is a list containing as first element the direct kinematics f (or None)
  and as second element the augmented jacobian (or None)
- r_dot is the task velocity
- values is a dictionary with keys the variables and values the values to give
  to those variables
- variables is a n-ple containing variables defined as symbols
"""

def task_augmentation(J, l, r_dot, values, variables):
  print("\nStarting J:\n", J)
  f = l[0]
  if f == None:
    J_a = l[1]
  else:
    J_a = sp.Matrix([sp.diff(f, var) for var in variables])

  print("\nAugmented J:\n", J_a)
  print("\nDesired velocity:\n", r_dot)

  check = input("\nIs J symbolic? (Y/N): ")
  if check == 'Y':
    J = np.round(np.array(J.subs(values)).astype(np.float64), 3)

  check = input("\nIs J_a symbolic? (Y/N): ")
  if check == 'Y':
    J_a = np.round(np.array(J_a.subs(values)).astype(np.float64), 3)

  J_final = np.vstack((J, J_a))
  print("\nFinal J:\n", J_final)

  m,n = J_final.shape
  if m==n:
    det = np.linalg.det(J_final)
    if det != 0:
      print("\nThe final jacobian is non-singular...\n")
      q_dot = np.dot(np.linalg.inv(J_final), r_dot)
      print("\nThe solution q_dot is:\n", q_dot)
  else:
    J_p = np.linalg.pinv(J_final)
    q_dot = np.dot(J_p, r_dot)
    print("\nThe final Jacobian is singular...\n")
    print("\nThe solution q_dot using the pseudoinverse is:\n", q_dot)

  return q_dot

In [ ]:
# example TA

l = sp.symbols('l')
s1 = sp.sin('q1')
s2 = sp.sin('q2')
s3 = sp.sin('q3')
s12 = sp.sin('q1+q2')
s123 = sp.sin('q1+q2+q3')
c1 = sp.cos('q1')
c2 = sp.cos('q2')
c3 = sp.cos('q3')
c12 = sp.cos('q1+q2')
c123 = sp.cos('q1+q2+q3')

J = sp.Matrix([
    [-l*(s1+s12+s123), -l*(s12+s123), -l*s123],
    [l*(c1+c12+c123), l*(c12+c123), l*c123]
])

J_a = sp.Matrix([
    [-3*l*(c1+c12), -2*l**2*s2-3*l*c12, 0],
])

l = [None, J_a]

values = {'l':1, 'q1':np.pi/2, 'q2':np.pi/3, 'q3':-2*np.pi/3}
q1 = sp.symbols('q1')
q2 = sp.symbols('q2')
q3 = sp.symbols('q3')
variables = (q1, q2, q3)

r_dot = np.array([[1], [-np.sqrt(3)], [0]])
q_dot = task_augmentation(J, l, r_dot, values, variables)


Starting J:
 Matrix([[-l*(sin(q1) + sin(q1 + q2) + sin(q1 + q2 + q3)), -l*(sin(q1 + q2) + sin(q1 + q2 + q3)), -l*sin(q1 + q2 + q3)], [l*(cos(q1) + cos(q1 + q2) + cos(q1 + q2 + q3)), l*(cos(q1 + q2) + cos(q1 + q2 + q3)), l*cos(q1 + q2 + q3)]])

Augmented J:
 Matrix([[-3*l*(cos(q1) + cos(q1 + q2)), -2*l**2*sin(q2) - 3*l*cos(q1 + q2), 0]])

Desired velocity:
 [[ 1.        ]
 [-1.73205081]
 [ 0.        ]]

Is J symbolic? (Y/N): Y

Is J_a symbolic? (Y/N): Y

Final J:
 [[-2.    -1.    -0.5  ]
 [ 0.     0.     0.866]
 [ 2.598  0.866  0.   ]]

The final jacobian is non-singular...


The solution q_dot is:
 [[-2.93346241e-05]
 [ 8.80038722e-05]
 [-2.00005867e+00]]


In [ ]:
"""
J1 and J2 are assumed to be numerically defined
as well as r1_dot, r2_dot and v2
"""
def TA_priority(J1, J2, r1_dot, r2_dot, v2, values):
  check = input("\nIs J1 symbolic? (Y/N): ")
  if check == 'Y':
    J1 = np.round(np.array(J1.subs(values)).astype(np.float64), 3)

  check = input("\nIs J2 symbolic? (Y/N): ")
  if check == 'Y':
    J2 = np.round(np.array(J2.subs(values)).astype(np.float64), 3)

  J1_p = np.linalg.pinv(J1)
  J2_p = np.linalg.pinv(J2)
  m,n = J1.shape

  I = np.identity(n)
  P1 = I-np.dot(J1_p, J1)
  J2P1 = np.dot(J2, P1)
  J2P1_p = np.linalg.pinv(np.dot(J2, P1))
  v1_1 = np.dot(J2P1_p, r2_dot-np.dot(np.dot(J2, J1_p), r1_dot))
  v1_2 = np.dot(I-np.dot(J2P1_p, J2P1), v2)
  v1 = v1_1+v1_2
  q_dot = np.dot(J1_p, r1_dot) + np.dot(P1, v1)
  print("\nThe solution q_dot is:\n", q_dot)
  return q_dot

In [ ]:
# example TA with priority

# example TA

l = sp.symbols('l')
s1 = sp.sin('q1')
s2 = sp.sin('q2')
s3 = sp.sin('q3')
s12 = sp.sin('q1+q2')
s123 = sp.sin('q1+q2+q3')
c1 = sp.cos('q1')
c2 = sp.cos('q2')
c3 = sp.cos('q3')
c12 = sp.cos('q1+q2')
c123 = sp.cos('q1+q2+q3')

J1 = sp.Matrix([
    [-l*(s1+s12+s123), -l*(s12+s123), -l*s123],
    [l*(c1+c12+c123), l*(c12+c123), l*c123]
])

J2 = sp.Matrix([
    [-3*l*(c1+c12), -2*l**2*s2-3*l*c12, 0],
    [-3*l*(s1+s12), -2*l**2*c2-3*l*s12, 0]
])


values = {'l':1, 'q1':np.pi/2, 'q2':np.pi/3, 'q3':-2*np.pi/3}
q1 = sp.symbols('q1')
q2 = sp.symbols('q2')
q3 = sp.symbols('q3')
variables = (q1, q2, q3)

r1_dot = np.array([[1], [-np.sqrt(3)]])
r2_dot = np.array([[1], [-np.sqrt(3)]])


q_dot = TA_priority(J1, J2, r1_dot, r2_dot, 0, values)


Is J1 symbolic? (Y/N): Y

Is J2 symbolic? (Y/N): Y

The solution q_dot is:
 [[ 0.43717602  0.43717602  0.43717602]
 [-0.12055827 -0.12055827 -0.12055827]
 [-1.95788775 -1.95788775 -1.95788775]]


In [ ]:
# TO DO: generalized (recursive) version of task priority

# Redundancy: acceleration level

In [ ]:
def acceleration_redundancy(J, J_dot, q, q_dot, q_double_dot_zero, r_doubledot, values, variables):
  print("\nStarting J:\n", J)
  print("\nStarting q:\n", q)
  print("\nStarting q_dot:\n", q_dot)
  print("\nPreferred acceleration:\n", q_double_dot_zero)

  check_J = input("Is J symbolic? (Y/N): ")
  if check_J=='Y':
    J = np.round(np.array(J.subs(values)).astype(np.float64), 3)

  m,n = J.shape
  x_double_dot = r_doubledot - np.dot(J_dot, q_dot)
  I = np.identity(m)
  J_p = np.linalg.pinv(J)
  projection = I-np.dot(J_p, J)

  # solution
  q_double_dot = np.dot(J_p, x_double_dot) + np.dot(projection, q_double_dot_zero)
  print("\nThe solution q_double_dot is:\n", q_double_dot)
  return q_double_dot

In [ ]:
# TO DO
def dynamic_redundancy_res():
  # first method
  typ = input("Do you want to solve with minimum torque norm solution? (Y/n): ")
  if typ == 'Y':
    pass
  else:
    # second method
    typ = input("Do you want to solve with minimum torque norm solution using Squared Inverse Inertia Matrix? (Y/n): ")
    if typ == 'Y':
      pass
    else:
      # third method
      typ = input("Do you want to solve with minimum torque norm solution using Inverse Inertia Matrix? (Y/n): ")
      if typ == 'Y':
        pass

  return

# SNS

In [ ]:
# this solution doesn't provide the scaling factor
# it stops the search in the null space when there's no violeted boundary

def SNS(J, q_bounds, v):

  print("\nWARNING: Indexes rely on the updated version of J and q_dot, not the original one!!!")

  print("\nStarting J:\n", J)
  print("\nVelocity bounds:\n", q_bounds)
  print("\nPreferred velocity:\n", v)

  m,n = J.shape

  v_new = np.zeros((m,1))
  q_dot = np.dot(np.linalg.pinv(J), v)

  fault = True

  while fault != False:
    errors = []

    print("\nq_dot:\n", q_dot)

    for i in range(len(q_bounds)):
      if i < len(q_bounds):
        q_max = q_bounds[i]
        q_min = -q_bounds[i]
        if q_dot[i]>q_max:
          errors.append(i+1)
          print(f"\nJoint {i+1} disrespects the max bounds...\nJOINT {i+1} will have velocity {q_bounds[i+1]}")
          q_bounds.pop(i)
          Ji = J[:,i:i+1]
          v_new = v - Ji * q_max
          print("\nNew velocity v =\n", v_new)
          J_i = np.delete(J, i, axis = 1)
          q_dot_new = np.dot(np.linalg.pinv(J_i), v_new)
          print("\nNew q_dot:\n", q_dot_new)

        elif q_dot[i] < q_min:
          errors.append(i+1)
          print(f"\nJoint {i+1} disrespects the min bounds...\nJOINT {i+1} will have velocity {q_bounds[i+1]}")
          q_bounds.pop(i)
          Ji = J[:,i:i+1]
          v_new = v - Ji * q_min
          print("\nNew velocity v =\n", v_new)
          J_i = np.delete(J, i, axis = 1)
          q_dot_new = np.dot(np.linalg.pinv(J_i), v_new)
          print("\nNew q_dot:\n", q_dot_new)


    if len(errors) == 0:
      print("\nNo other violations...\nEXIT")
      fault = False


    v = v_new
    q_dot = q_dot_new
    J = J_i

  return

In [ ]:
# midterm 29-03-2017
J = np.array([
    [0,0, 0, 0],
    [2, 1.5, 1, 0.5]
])


v = np.array([
    [0],
    [10]
])

q_bounds = [4, 2, 1, 1]

sol = SNS(J, q_bounds, v)



Starting J:
 [[0.  0.  0.  0. ]
 [2.  1.5 1.  0.5]]

Velocity bounds:
 [4, 2, 1, 1]

Preferred velocity:
 [[ 0]
 [10]]

q_dot:
 [[2.66666667]
 [2.        ]
 [1.33333333]
 [0.66666667]]

Joint 3 disrespects the max bounds...
JOINT 3 will have velocity 1

New velocity v =
 [[0.]
 [9.]]

New q_dot:
 [[2.76923077]
 [2.07692308]
 [0.69230769]]

q_dot:
 [[2.76923077]
 [2.07692308]
 [0.69230769]]

Joint 2 disrespects the max bounds...
JOINT 2 will have velocity 1

New velocity v =
 [[0.]
 [6.]]

New q_dot:
 [[2.82352941]
 [0.70588235]]

q_dot:
 [[2.82352941]
 [0.70588235]]

No other violations...
EXIT


In [ ]:
# exam 5-06-2020
J = np.array([
    [-1,-1, -0.5],
    [-0.366, -0.866, -0.866]
])


v = np.array([
    [2],
    [1]
])


q_bounds = [1,1.5, 2]

sol = SNS(J, q_bounds, v)



Starting J:
 [[-1.    -1.    -0.5  ]
 [-0.366 -0.866 -0.866]]

Velocity bounds:
 [1, 1.5, 2]

Preferred velocity:
 [[2]
 [1]]

q_dot:
 [[-1.33571392]
 [-0.73835425]
 [ 0.14813635]]

Joint 1 disrespects the min bounds...
JOINT 1 will have velocity 1.5

New velocity v =
 [[1.   ]
 [0.634]]

New q_dot:
 [[-1.26789838]
 [ 0.53579677]]

q_dot:
 [[-1.26789838]
 [ 0.53579677]]

No other violations...
EXIT


# DH: utils

In [ ]:
# this function creates a simulation of the table
"""
- alpha is a list of values alpha
- a is a list of values a
- d is a list of values/variables d
- theta is a list of values theta
"""

def create_DH_table(alpha, a, d, theta):
  dh = {'alpha': alpha, 'a': a, 'd': d, 'theta': theta}
  for i in range(len(dh['alpha'])):
    print(f"i = {i+1}\nalpha = {dh['alpha'][i]}\na = {dh['a'][i]}\nd = {dh['d'][i]}\ntheta = {dh['theta'][i]}")
    print()
  return dh

In [ ]:
# example of usage
dh = create_DH_table([-np.pi/2, np.pi/2, 0], [0,0,sp.symbols('dc3')],[sp.symbols('q1'),sp.symbols('l2'),0], [0, sp.symbols('q2'), sp.symbols('q3')])

i = 1
alpha = -1.5707963267948966
a = 0
d = q1
theta = 0

i = 2
alpha = 1.5707963267948966
a = 0
d = l2
theta = q2

i = 3
alpha = 0
a = dc3
d = 0
theta = q3



In [ ]:
# example of usage
dh = create_DH_table([np.pi/2, 0, 0], [0,sp.symbols('a2'),sp.symbols('a3')],[0,0,0], [sp.symbols('q1'), sp.symbols('q2'), sp.symbols('q3')])

i = 1
alpha = 1.5707963267948966
a = 0
d = 0
theta = q1

i = 2
alpha = 0
a = a2
d = 0
theta = q2

i = 3
alpha = 0
a = a3
d = 0
theta = q3



In [ ]:
# example of usage
dh = create_DH_table([0, 0], [0,sp.symbols('d2')],[sp.symbols('q1'), 0], [0, sp.symbols('q2')])

i = 1
alpha = 0
a = 0
d = q1
theta = 0

i = 2
alpha = 0
a = d2
d = 0
theta = q2



In [ ]:
# example of usage
dh = create_DH_table([0, 0, 0, 0], [sp.symbols('l1'),sp.symbols('l2'),sp.symbols('l3'), sp.symbols('l4')],[0, 0,0,0], [sp.symbols('q1'), sp.symbols('q2'), sp.symbols('q3'),sp.symbols('q4')])

i = 1
alpha = 0
a = l1
d = 0
theta = q1

i = 2
alpha = 0
a = l2
d = 0
theta = q2

i = 3
alpha = 0
a = l3
d = 0
theta = q3

i = 4
alpha = 0
a = l4
d = 0
theta = q4



In [ ]:
# this function creates the matrix with alpha_i, a_i, d_i, theta_i
def create_DH_matrix(i, alpha, a, d, theta):
  threshold = 1e-10

  A_00 = sp.cos(theta)
  A_010 = sp.cos(alpha)
  A_011 = sp.sin(theta)
  A_020 = sp.sin(alpha)
  A_021 = sp.sin(theta)
  A_120 = sp.cos(alpha)

  if A_00.is_number:
    if A_00 < threshold:
      A_00 = 0

  if A_010.is_number:
    if abs(A_010) < threshold:
      A_010 = 0

  if A_011.is_number:
    if abs(A_011) < threshold:
      A_011 = 0

  if A_020.is_number:
    if abs(A_020) < threshold:
      A_020 = 0


  if A_021.is_number:
    if abs(A_021) < threshold:
      A_021 = 0

  if A_120.is_number:
    if abs(A_120) < threshold:
      A_120 = 0

  A_i = np.array([
      [A_00, -A_010*A_011, A_020*A_021, sp.simplify(a)*A_00],
      [A_011, A_010*A_00, -A_020*A_00, sp.simplify(a)*A_011],
      [0, A_020, A_120, sp.simplify(d)],
      [0, 0, 0, 1]
  ])

  R_i = A_i[:3, :3]
  last_column = A_i[:,-1]
  r_i = last_column[:3].reshape(-1,1)

  print(A_i)
  print()
  return A_i, R_i, r_i

def define_all_matrices(N_joints, dh):
  A = []
  R = []
  r = []
  for i in range(N_joints):
    print(f"\nMatrix A associated to RF{i+1} wrt RF{i}\n")
    A_i, R_i, r_i = create_DH_matrix(i, dh['alpha'][i], dh['a'][i], dh['d'][i], dh['theta'][i])
    A.append(A_i)
    R.append(R_i)
    r.append(r_i)
  return A, R, r

In [ ]:
# example: determine each A of frames
A, R, r = define_all_matrices(3, dh)


Matrix A associated to RF1 wrt RF0

[[1 0 0 0]
 [0 0 1.00000000000000 0]
 [0 -1.00000000000000 0 q1]
 [0 0 0 1]]


Matrix A associated to RF2 wrt RF1

[[cos(q2) 0 1.0*sin(q2) 0]
 [sin(q2) 0 -1.0*cos(q2) 0]
 [0 1.00000000000000 0 l2]
 [0 0 0 1]]


Matrix A associated to RF3 wrt RF2

[[cos(q3) -sin(q3) 0 dc3*cos(q3)]
 [sin(q3) cos(q3) 0 dc3*sin(q3)]
 [0 0 1 0]
 [0 0 0 1]]



In [ ]:
# example: determine each A of frames
A, R, r = define_all_matrices(3, dh)


Matrix A associated to RF1 wrt RF0

[[cos(q1) 0 1.0*sin(q1) 0]
 [sin(q1) 0 -1.0*cos(q1) 0]
 [0 1.00000000000000 0 0]
 [0 0 0 1]]


Matrix A associated to RF2 wrt RF1

[[cos(q2) -sin(q2) 0 a2*cos(q2)]
 [sin(q2) cos(q2) 0 a2*sin(q2)]
 [0 0 1 0]
 [0 0 0 1]]


Matrix A associated to RF3 wrt RF2

[[cos(q3) -sin(q3) 0 a3*cos(q3)]
 [sin(q3) cos(q3) 0 a3*sin(q3)]
 [0 0 1 0]
 [0 0 0 1]]



In [ ]:
# example: determine each A of frames
A, R, r = define_all_matrices(2, dh)


Matrix A associated to RF1 wrt RF0

[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 q1]
 [0 0 0 1]]


Matrix A associated to RF2 wrt RF1

[[cos(q2) -sin(q2) 0 d2*cos(q2)]
 [sin(q2) cos(q2) 0 d2*sin(q2)]
 [0 0 1 0]
 [0 0 0 1]]



In [ ]:
# example: determine each A of frames
A, R, r = define_all_matrices(4, dh)


Matrix A associated to RF1 wrt RF0

[[cos(q1) -sin(q1) 0 l1*cos(q1)]
 [sin(q1) cos(q1) 0 l1*sin(q1)]
 [0 0 1 0]
 [0 0 0 1]]


Matrix A associated to RF2 wrt RF1

[[cos(q2) -sin(q2) 0 l2*cos(q2)]
 [sin(q2) cos(q2) 0 l2*sin(q2)]
 [0 0 1 0]
 [0 0 0 1]]


Matrix A associated to RF3 wrt RF2

[[cos(q3) -sin(q3) 0 l3*cos(q3)]
 [sin(q3) cos(q3) 0 l3*sin(q3)]
 [0 0 1 0]
 [0 0 0 1]]


Matrix A associated to RF4 wrt RF3

[[cos(q4) -sin(q4) 0 l4*cos(q4)]
 [sin(q4) cos(q4) 0 l4*sin(q4)]
 [0 0 1 0]
 [0 0 0 1]]



In [ ]:
"""
here there are some useful function to manage the homogenous matrices
"""
def compute_T(R, p):
  r11 = R[0][0]
  r12 = R[0][1]
  r13 = R[0][2]
  r21 = R[1][0]
  r22 = R[1][1]
  r23 = R[1][2]
  r31 = R[2][0]
  r32 = R[2][1]
  r33 = R[2][2]

  p1 = p[0]
  p2 = p[1]
  p3 = p[2]

  T = np.array([
      [r11, r12, r13, p1],
      [r21, r22, r23, p2],
      [r31, r32, r33, p3],
      [0, 0, 0, 1]
  ])

  print("T matrix:\n", T)
  return T


def multiplications_of_T(T1, T2):
  dim_T1 = T1.shape
  dim_T2 = T2.shape
  if (dim_T1 == dim_T2 and dim_T1[0]==4 and dim_T1[1] == 4) == False:
    return "Invalid dimensions"
  else:
    T_res = np.dot(T1, T2)
    return T_res

def composition_of_T(list_of_T, Base, v):
  if v==True:
    list_of_T = [Base]+list_of_T  # added the multiplication with a base frame hom. matrix
  n = len(list_of_T)
  if n>0:
    result = np.eye(list_of_T[0].shape[0])
    for i in range(n):
      result = multiplications_of_T(result, list_of_T[i])
      result = sp.simplify(result)
    for row in result:
      print(sp.simplify(row))
    return result
  else:
    return "Empty List"

In [ ]:
# determine the result (chain of multiplications)

result = composition_of_T(A, [], False)

[1.0*cos(q2)*cos(q3), -1.0*sin(q3)*cos(q2), 1.0*sin(q2), 1.0*dc3*cos(q2)*cos(q3)]
[1.0*sin(q3), 1.0*cos(q3), 0, 1.0*dc3*sin(q3) + 1.0*l2]
[-1.0*sin(q2)*cos(q3), 1.0*sin(q2)*sin(q3), 1.0*cos(q2), -1.0*dc3*sin(q2)*cos(q3) + 1.0*q1]
[0, 0, 0, 1.0]


In [ ]:
# determine the result (chain of multiplications)

result = composition_of_T(A, [], False)

[1.0*cos(q1)*cos(q2 + q3), -1.0*sin(q2 + q3)*cos(q1), 1.0*sin(q1), 1.0*(a2*cos(q2) + a3*cos(q2 + q3))*cos(q1)]
[1.0*sin(q1)*cos(q2 + q3), -1.0*sin(q1)*sin(q2 + q3), -1.0*cos(q1), 1.0*(a2*cos(q2) + a3*cos(q2 + q3))*sin(q1)]
[1.0*sin(q2 + q3), 1.0*cos(q2 + q3), 0, 1.0*a2*sin(q2) + 1.0*a3*sin(q2 + q3)]
[0, 0, 0, 1.0]


In [ ]:
# determine the result (chain of multiplications)

result = composition_of_T(A, [], False)

[1.0*cos(q1 + q2), -1.0*sin(q1 + q2), 0, 1.0*a1*cos(q1) + 1.0*a2*cos(q1 + q2)]
[1.0*sin(q1 + q2), 1.0*cos(q1 + q2), 0, 1.0*a1*sin(q1) + 1.0*a2*sin(q1 + q2)]
[0, 0, 1.0, 0]
[0, 0, 0, 1.0]


In [ ]:
# insert values inside Result
values = {'q1':0, 'q2':np.pi/6, 'q3':-np.pi/2}
R_valued = result.subs(values)

for row in R_valued:
  print(row)

[0.5, 0.866025403784439, 0, 0.866025403784439*a2 + 0.5*a3]
[0, 0, -1.0, 0]
[-0.866025403784439, 0.5, 0, 0.5*a2 - 0.866025403784439*a3]
[0, 0, 0, 1.0]


# Jacobian derivative

In [ ]:
def compute_Jdot(J, variables):

  print("Jacobian:\n")
  display(Math(sp.latex(sp.simplify(J))))

  m = J.shape[0]
  n = J.shape[1]

  j = sp.zeros(m,n)
  for i in range(len(variables)):
    q_i = variables[i]
    q_i_dot = sp.symbols('q'+str(i+1)+'_dot')
    j += sp.diff(J, q_i)*q_i_dot

  print("\nJ_dot * q_dot:\n")
  display(Math(sp.latex(sp.simplify(j))))


  return


In [ ]:
c1 = sp.cos('q1')
s1 = sp.sin('q1')
c12 = sp.cos('q1+q2')
s12 = sp.sin('q1+q2')
l = sp.symbols('L')
J = sp.Matrix([
    [l*(s1+s12), l*s12],
    [l*(c1+c12), l*c12]
])

variables = [sp.symbols('q1'), sp.symbols('q2')]
compute_Jdot(J, variables)

Jacobian:



<IPython.core.display.Math object>


J_dot * q_dot:



<IPython.core.display.Math object>

# Euler-Lagrange Method

In [ ]:
"""
p_c is the vector of elements with positions of center of mass i wrt RF0
w is the vector of angular velocities associated to each link
variables is the list of var for which we have to differentiate
"""

def kinematic_energy(p_c, w, m, I, variables, derivatives):
  n = len(p_c)
  T = 0
  for i in range(n):
    v_ci = sp.zeros(3,1)
    p_ci = p_c[i]
    J = np.array(p_ci.jacobian(variables))

    for r in range(J.shape[0]):
      v_cic = 0
      for c in range(J.shape[1]):
        v_cic += J[r][c]*derivatives[c]
      v_ci[r]=v_cic


    print(f"\nPosition COM{i+1}:\n")
    display(Math(sp.latex(p_ci)))

    print(f"\nVelocity COM{i+1}:\n")
    display(Math(sp.latex(v_ci)))

    T_i_w = np.array(sp.simplify(0.5*I[i]*w[i]*w[i]))
    T_i_vc = np.array(sp.simplify(0.5*m[i]*(v_ci.T*v_ci)))
    T_i = sp.simplify(T_i_w + T_i_vc)[0][0]
    print(f"\nKinetic energy of link {i+1}:\n")
    display(Math(sp.latex(T_i)))

    T += T_i

  M = sp.hessian(T, (sp.symbols('q1_dot'), sp.symbols('q2_dot'),sp.symbols('q3_dot'), sp.symbols('q4_dot')))
  M_latex = sp.latex(sp.simplify(M))
  print("\nInertia Matrix:\n")
  display(Math(M_latex))

  return

In [ ]:
m1 = sp.symbols('m1')
m2 = sp.symbols('m2')
m3 = sp.symbols('m3')
m4 = sp.symbols('m4')

d1 = sp.symbols('d1')
d2 = sp.symbols('d2')
d3 = sp.symbols('d3')
d4 = sp.symbols('d4')

l1 = sp.symbols('l1')

q1 = sp.symbols('q1')
q2 = sp.symbols('q2')
q3 = sp.symbols('q3')
q4 = sp.symbols('q4')

q1_dot = sp.symbols('q1_dot')
q2_dot = sp.symbols('q2_dot')
q3_dot = sp.symbols('q3_dot')
q4_dot = sp.symbols('q4_dot')

c1 = sp.cos('q1')
s1 = sp.sin('q1')
s12 = sp.sin('q1+q2')
c12 = sp.cos('q1+q2')
c13 = sp.cos('q1+q3')
s13 = sp.sin('q1+q3')
c124 = sp.cos('q1+q2+q4')
s124 = sp.sin('q1+q2+q4')

I1 = sp.symbols('I1')
I2 = sp.symbols('I2')
I3 = sp.symbols('I3')
I4 = sp.symbols('I4')

p_c1 = sp.Matrix([
    [d1*c1],
    [d1*s1],
    [0]
])

p_c2 = sp.Matrix([
    [l1*c1+s12],
    [l1*s1+c12],
    [0]
])

p_c3 = sp.Matrix([
    [l1*c1+(q3-d3)*c12],
    [l1*s1+(q3-d3)*s12],
    [0]
])

p_c4 = sp.Matrix([
    [l1*c1+q3*c12+d4*c124],
    [l1*s1+q3*s12+d4*s124],
    [0]
])

w1 = q1_dot
w2 = q2_dot
w3 = q1_dot+q2_dot
w4 = q1_dot+q2_dot+q4_dot

p_c = [p_c1, p_c2, p_c3, p_c4]
w = [w1, w2, w3, w4]
m = [m1, m2, m3, m4]
I = [I1, I2, I3, I4]
kinematic_energy(p_c, w, m, I, [q1,q2,q3,q4], [q1_dot,q2_dot,q3_dot,q4_dot])


Position COM1:



<IPython.core.display.Math object>


Velocity COM1:



<IPython.core.display.Math object>


Kinetic energy of link 1:



...

<IPython.core.display.Math object>


Kinetic energy of link 4:



<IPython.core.display.Math object>


Inertia Matrix:



<IPython.core.display.Math object>

## Moving Frames

In [ ]:
"""
- R is the list of rotation matrices
- r is the positional vector coming from frame i-1 to i
- r_c is a list of positional vector from RF_i to the COM_i
- m is a list of symbolic variables m_i
- I_c is a list of symbolic variables I_ci,zz
- sigma is a list of 0 or 1 (1 if the joint i is P, 0 else)
- q_dot is a list of symbolic variables q_dot_i
"""
def moving_frames(R, r, r_c, m, I_c, sigma, q_dot):
  sp.init_printing()
  w_0 = np.zeros((3,1))
  v_0 = np.zeros((3,1))
  z_prev = np.array([[0], [0], [1]])
  T = 0
  for i in range(len(R)):
    R_i = R[i]          # rotation matrix ^{i-1}RF_i
    print(f"\nRotation matrix from {i} to {i+1}:\n")
    display(Math(sp.latex(sp.Matrix(R_i).applyfunc(sp.simplify))))
    r_i = np.array(sp.simplify(np.dot(R_i.T,r[i]))) # moving frame
    print(f"\nPositional vector between origins in moving frame:\n")
    display(Math(sp.latex(sp.Matrix(r_i).applyfunc(sp.simplify))))
    r_ci = r_c[i]
    print(f"\nPositional vector of link CoM{i} in moving frame:\n")
    display(Math(sp.latex(sp.Matrix(r_ci).applyfunc(sp.simplify))))

    if i == 0:
      w_i = np.dot(R_i.T, w_0+(1-sigma[i])*q_dot[i]*z_prev)

      t1 = np.dot(R_i.T, v_0)
      t2 = np.dot(R_i.T, sigma[i]*q_dot[i]*z_prev)
      t3 = np.cross(w_i.flatten(), r_i.flatten()).reshape(-1,1)
      v_i = t1+t2+t3


    else:
      w_i = np.dot(R_i.T, w_i_prev+(1-sigma[i])*q_dot[i]*z_prev)

      t1 = np.dot(R_i.T, v_i_prev)
      t2 = np.dot(R_i.T, sigma[i]*q_dot[i]*z_prev)
      t3 = np.cross(w_i.flatten(), r_i.flatten()).reshape(-1,1)
      v_i = t1+t2+t3

    v_i_prev = v_i
    w_i_prev = w_i

    v_ci = v_i + np.cross(w_i.flatten(), r_ci.flatten()).reshape(-1,1)
    w_i_latex = sp.latex(sp.Matrix(w_i).applyfunc(sp.simplify))
    v_i_latex = sp.latex(sp.Matrix(v_i).applyfunc(sp.simplify))
    v_ci_latex = sp.latex(sp.Matrix(v_ci).applyfunc(sp.simplify))
    T_i = 0.5*m[i]*np.dot(v_ci.T, v_ci)+0.5*np.dot(w_i.T, w_i)*I_c[i]
    T += T_i
    T_i_latex = sp.latex(sp.Matrix(T_i).applyfunc(sp.simplify))
    print(f"\ni = {i+1}")
    print(f"\nw_{i+1}:\n")
    display(Math(w_i_latex))
    print(f"\nv_{i+1}:\n")
    display(Math(v_i_latex))
    print(f"\nv_c{i+1}:\n")
    display(Math(v_ci_latex))
    print(f"\nT_{i+1}:\n")
    display(Math(T_i_latex))

  return T

In [ ]:
sigma = [1,0,0]
q_dot = [sp.symbols('q_dot_1'), sp.symbols('q_dot_2'), sp.symbols('q_dot_3')]
r_c = [np.array([[0], [sp.symbols('dc1')], [0]]), np.array([[0], [sp.symbols('dc2')-sp.symbols('l2')], [0]]), np.array([[sp.symbols('dc3')], [0], [0]])]
m = [sp.symbols('m1'), sp.symbols('m2'), sp.symbols('m3')]
I_c = [sp.symbols('I_c1'), sp.symbols('I_c2'), sp.symbols('I_c3')]
T = moving_frames(R, r, r_c, m, I_c, sigma, q_dot)

print("\nInertia Matrix:\n")
M = sp.hessian(T.item(), (sp.symbols('q_dot_1'), sp.symbols('q_dot_2'),sp.symbols('q_dot_3')))
M_latex = sp.latex(sp.simplify(M))
display(Math(M_latex))



Rotation matrix from 0 to 1:



<IPython.core.display.Math object>


Positional vector between origins in moving frame:



<IPython.core.display.Math object>


Positional vector of link CoM0 in moving frame:



...

<IPython.core.display.Math object>


v_c3:



<IPython.core.display.Math object>


T_3:



<IPython.core.display.Math object>

In [ ]:
sigma = [1,0] # 0 : joint revolute, 1: joint prismatic
q_dot = [sp.symbols('q_dot_1'), sp.symbols('q_dot_2')]
r_c = [np.array([[sp.symbols('q1')], [0], [0]]), np.array([[sp.symbols('q1')+sp.symbols('d2')*sp.cos('q2')], [sp.symbols('d2')*sp.sin('q2')], [0]])]
m = [sp.symbols('m1'), sp.symbols('m2')]
I_c = [sp.symbols('I_c1'), sp.symbols('I_c2')]
T = moving_frames(R, r, r_c, m, I_c, sigma, q_dot)

print("\nInertia Matrix:\n")
M = sp.hessian(T.item(), (sp.symbols('q_dot_1'), sp.symbols('q_dot_2')))
M_latex = sp.latex(sp.simplify(M))
display(Math(M_latex))



Rotation matrix from 0 to 1:



<IPython.core.display.Math object>


Positional vector between origins in moving frame:



<IPython.core.display.Math object>


Positional vector of link CoM0 in moving frame:



...

<IPython.core.display.Math object>


T_2:



<IPython.core.display.Math object>


Inertia Matrix:



<IPython.core.display.Math object>

In [ ]:
sigma = [0,0,0]
q_dot = [sp.symbols('q_dot_1'), sp.symbols('q_dot_2'), sp.symbols('q_dot_3')]
r_c = [np.array([[0], [sp.symbols('dc1')], [0]]), np.array([[sp.symbols('dc2')-sp.symbols('a2')], [0], [0]]), np.array([[sp.symbols('dc3')-sp.symbols('a3')], [0], [0]])]
m = [sp.symbols('m1'), sp.symbols('m2'), sp.symbols('m3')]
I_c = [sp.symbols('I_c1'), sp.symbols('I_c2'), sp.symbols('I_c3')]
moving_frames(R, r, r_c, m, I_c, sigma, q_dot)


print("\nInertia Matrix:\n")
M = sp.hessian(T.item(), (sp.symbols('q_dot_1'), sp.symbols('q_dot_2'),sp.symbols('q_dot_3')))
M_latex = sp.latex(sp.simplify(M))
display(Math(M_latex))


Rotation matrix from 0 to 1:



<IPython.core.display.Math object>


Positional vector between origins in moving frame:



<IPython.core.display.Math object>


Positional vector of link CoM0 in moving frame:



...

<IPython.core.display.Math object>


v_c3:



<IPython.core.display.Math object>


T_3:



<IPython.core.display.Math object>

In [ ]:
sigma = [0,0,0,0]
q_dot = [sp.symbols('q_dot_1'), sp.symbols('q_dot_2'), sp.symbols('q_dot_3'),sp.symbols('q_dot_4')]
r_c = [np.array([[sp.symbols('dc1')-sp.symbols('l1')], [0], [0]]), np.array([[sp.symbols('dc2')-sp.symbols('l2')], [0], [0]]), np.array([[sp.symbols('dc3')-sp.symbols('l3')], [0], [0]]), np.array([[0], [0], [0]])]
m = [sp.symbols('m1'), sp.symbols('m2'), sp.symbols('m3'), sp.symbols('m4')]
I_c = [sp.symbols('I_c1'), sp.symbols('I_c2'), sp.symbols('I_c3'), sp.symbols('I_c4')]
moving_frames(R, r, r_c, m, I_c, sigma, q_dot)


print("\nInertia Matrix:\n")
M = sp.hessian(T.item(), (sp.symbols('q_dot_1'), sp.symbols('q_dot_2'),sp.symbols('q_dot_3'),sp.symbols('q_dot_4')))
M_latex = sp.latex(sp.simplify(M))
display(Math(M_latex))


Rotation matrix from 0 to 1:



<IPython.core.display.Math object>


Positional vector between origins in moving frame:



<IPython.core.display.Math object>


Positional vector of link CoM0 in moving frame:



...

<IPython.core.display.Math object>


v_c4:



<IPython.core.display.Math object>


T_4:



<IPython.core.display.Math object>

## Centrifugal and Coriolis terms

In [ ]:
"""
- M is a quadratic matrix (symbolic)
- q_dot is a symbolic vector containing the symbolic variables of each joint velocity
- variables is a list of symbolic variables for which we have to differentiate
"""
def compute_centrifugal_coriolis(M, q_dot, variables):

  n = M.shape[0]
  C = []
  c = []

  for i in range(n):
    M_i = M[:,i]
    dMi_dq = M_i.jacobian(variables)
    dM_dqi = sp.Matrix([sp.diff(M, variables[i])])
    C_i = 0.5*(dMi_dq + dMi_dq.T - dM_dqi)
    C.append(C_i)
    c_i = q_dot.T * C_i * q_dot
    C_i_latex = sp.latex(C_i)
    c_i_latex = sp.latex(c_i)
    print(f"\nC_{i+1}:\n")
    display(Math(C_i_latex))
    print(f"\nc_{i+1}:\n")
    display(Math(c_i_latex))

  return

In [ ]:
# build your M matrix from the computations above
# example 2R planar robot

q1 = sp.symbols('q1')
q2 = sp.symbols('q2')
variables = [q1, q2]

A = sp.symbols('A')
B = sp.symbols('B')
C = sp.symbols('C')
c2 = sp.cos('q2')

m11 = A
m12 = B*c2
m21 = m12
m22 = C

M = sp.Matrix([
    [m11,m12],
    [m21,m22]
])

q_dot_1 = sp.symbols('q_dot_1')
q_dot_2 = sp.symbols('q_dot_2')
q_dot = sp.Matrix([
    [q_dot_1],
    [q_dot_2]
])

compute_centrifugal_coriolis(M, q_dot, variables)


C_1:



<IPython.core.display.Math object>


c_1:



<IPython.core.display.Math object>


C_2:



<IPython.core.display.Math object>


c_2:



<IPython.core.display.Math object>

In [ ]:
# build your M matrix from the computations above
# example 2R planar robot

q1 = sp.symbols('q1')
q2 = sp.symbols('q2')
variables = [q1, q2]

m1,m2 = sp.symbols('m1 m2')
Ic2 = sp.symbols('Ic2')
d2 = sp.symbols('d2')
c2 = sp.cos('q2')

m11 = m1+m2
m12 = d2*c2
m21 = m12
m22 = Ic2+d2**2

M = sp.Matrix([
    [m11,m12],
    [m21,m22]
])

q_dot_1 = sp.symbols('q_dot_1')
q_dot_2 = sp.symbols('q_dot_2')
q_dot = sp.Matrix([
    [q_dot_1],
    [q_dot_2]
])

compute_centrifugal_coriolis(M, q_dot, variables)


C_1:



<IPython.core.display.Math object>


c_1:



<IPython.core.display.Math object>


C_2:



<IPython.core.display.Math object>


c_2:



<IPython.core.display.Math object>

In [ ]:
# example 2R planar robot
q1 = sp.symbols('q1')
q2 = sp.symbols('q2')
variables = [q1, q2]

a1 = sp.symbols('a1')
a2 = sp.symbols('a2')
a3 = sp.symbols('a3')
c2 = sp.cos('q2')

m11 = a1+2*a2*c2
m12 = a3+a2*c2
m21 = m12
m22 = a3

M = sp.Matrix([
    [m11,m12],
    [m21,m22]
])

q_dot_1 = sp.symbols('q_dot_1')
q_dot_2 = sp.symbols('q_dot_2')
q_dot = sp.Matrix([
    [q_dot_1],
    [q_dot_2]
])

compute_centrifugal_coriolis(M, q_dot, variables)


C_1:



<IPython.core.display.Math object>


c_1:



<IPython.core.display.Math object>


C_2:



<IPython.core.display.Math object>


c_2:



<IPython.core.display.Math object>

In [ ]:
# example 2R planar robot
q1 = sp.symbols('q1')
q2 = sp.symbols('q2')
variables = [q1, q2]

m1 = sp.symbols('m1')
m2 = sp.symbols('m2')
dc2 = sp.symbols('d_c2')
s2 = sp.sin('q2')
Ic2 = sp.symbols('Ic2')

m11 = m1+m2
m12 = -m2*dc2*s2
m21 = -m2*dc2*s2
m22 = Ic2+m2*dc2**2

M = sp.Matrix([
    [m11,m12],
    [m21,m22]
])

q_dot_1 = sp.symbols('q_dot_1')
q_dot_2 = sp.symbols('q_dot_2')
q_dot = sp.Matrix([
    [q_dot_1],
    [q_dot_2]
])

compute_centrifugal_coriolis(M, q_dot, variables)


C_1:



<IPython.core.display.Math object>


c_1:



<IPython.core.display.Math object>


C_2:



<IPython.core.display.Math object>


c_2:



<IPython.core.display.Math object>

In [ ]:
# example 2R planar robot
q1 = sp.symbols('q1')
q2 = sp.symbols('q2')
q3 = sp.symbols('q3')
variables = [q1, q2, q3]

m1 = sp.symbols('m1')
m2 = sp.symbols('m2')
m3 = sp.symbols('m3')
dc3 = sp.symbols('d_c3')
s2 = sp.sin('q2')
s3 = sp.sin('q3')
c2 = sp.cos('q2')
c3 = sp.cos('q3')
Ic2 = sp.symbols('Ic2')
Ic3y = sp.symbols('Ic3y')
Ic3x = sp.symbols('Ic3x')
Ic3z = sp.symbols('Ic3z')

m11 = m1+m2+m3
m12 = -m3*dc3*c2*c3
m13 = m3*dc3*s2*s3

m21 = -m3*dc3*c2*c3
m22 = Ic2+Ic3x+(Ic3y-Ic3x+m3*dc3**2)*c3**2
m23 = 0

m31 = m3*dc3*s2*s3
m32 = 0
m33 = Ic3z+m3*dc3**2

M = sp.Matrix([
    [m11,m12,m13],
    [m21,m22,m23],
    [m31,m32,m33]
])

q_dot_1 = sp.symbols('q_dot_1')
q_dot_2 = sp.symbols('q_dot_2')
q_dot_3 = sp.symbols('q_dot_3')
q_dot = sp.Matrix([
    [q_dot_1],
    [q_dot_2],
    [q_dot_3]
])

compute_centrifugal_coriolis(M, q_dot, variables)


C_1:



<IPython.core.display.Math object>


c_1:



<IPython.core.display.Math object>


C_2:



...

<IPython.core.display.Math object>


C_3:



<IPython.core.display.Math object>


c_3:



<IPython.core.display.Math object>

## Gravity terms

In [ ]:
"""
- g is a symbolic vector containing for each component the gravity effect on that component
- r is a list of vectors containing the positional vectors from the origin to the CoM of that component
- m is a list of symbols m_i
- n is the number of joints
- variables is a list of variable to which differentiate U
"""
def compute_gravity(g, r, m, n, variables):
  U = np.zeros((1, 1))
  for i in range(n):
    m_i = m[i]
    r_i = r[i]
    u_i = -m_i*g.T*r_i
    U += u_i.T

  U_latex = sp.latex(U)
  print("\nTotal gravity U:\n")
  display(Math(U_latex))

  g_q = U.jacobian(variables)
  g_q_latex = sp.latex(g_q.T)
  print("\nGravity vector g(q):\n")
  display(Math(g_q_latex))

  return

In [ ]:
g0 = sp.symbols('g0')
g = sp.Matrix([
    [0],
    [-g0],
    [0]
])

d1 = sp.symbols('d1')
s1 = sp.sin('q1')
r1 = sp.Matrix([
    [0],
    [d1*s1],
    [0]
])

l1 = sp.symbols('l1')
d2 = sp.symbols('d2')
s12 = sp.sin('q1+q2')

r2 = sp.Matrix([
    [0],
    [l1*s1+d2*s12],
    [0]
])

r = [r1, r2]

m1 = sp.symbols('m1')
m2 = sp.symbols('m2')
m = [m1, m2]

n = 2
variables = [sp.symbols('q1'), sp.symbols('q2')]

compute_gravity(g, r, m, n, variables)


Total gravity U:



<IPython.core.display.Math object>


Gravity vector g(q):



<IPython.core.display.Math object>

# Newton-Euler Method

## Forward recursion

In [ ]:
"""
this function only works with revolute joints based robot
- q is the list of variables
- q_dot is a column vector containing the velocities for each joint
- q_ddot is a column vector containing the accelerations for each joint
- g is a vector of gravity terms
- r is a list of vectors of the positions ^ir_{i-1,i}
- r_c is a list of vectors of the positions ^ir_{i, ci}
"""
def NE_forward(q, q_dot, q_ddot, g, r, r_c):

  Wi = []
  Wi_dot = []
  Ai = []
  Aci = []

  # initialization step
  n = len(q)
  w0 = np.zeros((n,1))
  w0_dot = np.zeros((n,1))

  z0 = np.array([
       [0],
       [0],
       [1]
  ])

  a0 = -g

  # computation
  for i in range(1,n+1):
    print(f"\nJoint {i}")
    ci = sp.cos('q'+str(i))
    si = sp.sin('q'+str(i))
    Ri = sp.Matrix(
        [
            [ci, -si, 0],
            [si, ci, 0],
            [0, 0, 1]
        ]
    )

    if i == 1:
      wi = Ri.T * (w0 + q_dot[i-1] * z0)
      term = np.cross((q_dot[i-1].item()*w0).reshape(1,3), z0.reshape(1,3)).reshape(3,1)
      wi_dot = Ri.T * (w0_dot + q_ddot[i].item()*z0 + term)
      c1 = np.cross(wi_dot.reshape(1,3), r[i-1].reshape(1,3)).reshape(3,1)
      c2 = np.cross(wi.reshape(1,3), r[i-1].reshape(1,3))
      ai = Ri.T * a0 + c1 + np.cross(wi.reshape(1,3), c2).reshape(3,1)
      last_term = np.cross(wi.reshape(1,3), np.cross(wi.reshape(1,3), r_c[i-1].reshape(1,3))).reshape(3,1)
      aci = ai+np.cross(wi_dot.reshape(1,3), r_c[i-1].reshape(1,3)).reshape(3,1) + last_term
    else:
      wi = Ri.T * (wi + q_dot[i-1].item()* z0)
      term = np.cross((q_dot[i-1].item()*wi).reshape(1,3), z0.reshape(1,3)).reshape(3,1)
      wi_dot = Ri.T * (wi_dot + q_ddot[i-1].item()*z0 + term)
      c1 = np.cross(wi_dot.reshape(1,3), r[i-1].reshape(1,3)).reshape(3,1)
      c2 = np.cross(wi.reshape(1,3), r[i-1].reshape(1,3))
      ai = Ri.T * ai + c1 + np.cross(wi.reshape(1,3), c2).reshape(3,1)
      last_term = np.cross(wi.reshape(1,3), np.cross(wi.reshape(1,3), r_c[i-1].reshape(1,3))).reshape(3,1)
      aci = ai+np.cross(wi_dot.reshape(1,3), r_c[i-1].reshape(1,3)).reshape(3,1) + last_term

    Wi.append(wi)
    Wi_dot.append(wi_dot)
    Ai.append(ai)
    Aci.append(aci)

    wi_l = sp.latex(wi)
    wi_dot_l = sp.latex(wi_dot)
    ai_l = sp.latex(sp.simplify(ai))
    aci_l = sp.latex(sp.simplify(aci))
    print(f"\nw_{i}:\n")
    display(Math(wi_l))
    print(f"\nw_{i}_dot:\n")
    display(Math(wi_dot_l))
    print(f"\na_{i}:\n")
    display(Math(ai_l))
    print(f"\na_c{i}:\n")
    display(Math(aci_l))

  return Wi, Wi_dot, Ai, Aci

In [ ]:
# example of usage
dh = create_DH_table([0, 0, 0], [sp.symbols('a1'),sp.symbols('a2'),sp.symbols('a2')],[0,0,0], [sp.symbols('q1'), sp.symbols('q2'),sp.symbols('q3')])
A,R,r = define_all_matrices(3, dh)
q = [sp.symbols('q1'), sp.symbols('q2'), sp.symbols('q3')]
n = len(q)
q_dot = np.zeros((n,1))
q_ddot = np.zeros((n,1))
g = sp.Matrix([
    [sp.symbols('g0')],
    [0],
    [0]
])

r_c = [np.array([[sp.symbols('d1')], [0], [0]]), np.array([[sp.symbols('d2')], [0], [0]]), np.array([[sp.symbols('d3')], [0], [0]])]


Wi, Wi_dot, Ai, Aci = NE_forward(q, q_dot, q_ddot, g, r, r_c)

i = 1
alpha = 0
a = a1
d = 0
theta = q1

i = 2
alpha = 0
a = a2
d = 0
theta = q2

i = 3
alpha = 0
a = a2
d = 0
theta = q3


Matrix A associated to RF1 wrt RF0

[[cos(q1) -sin(q1) 0 a1*cos(q1)]
 [sin(q1) cos(q1) 0 a1*sin(q1)]
 [0 0 1 0]
 [0 0 0 1]]


Matrix A associated to RF2 wrt RF1

[[cos(q2) -sin(q2) 0 a2*cos(q2)]
 [sin(q2) cos(q2) 0 a2*sin(q2)]
 [0 0 1 0]
 [0 0 0 1]]


Matrix A associated to RF3 wrt RF2

[[cos(q3) -sin(q3) 0 a2*cos(q3)]
 [sin(q3) cos(q3) 0 a2*sin(q3)]
 [0 0 1 0]
 [0 0 0 1]]


Joint 1

w_1:



<IPython.core.display.Math object>


w_1_dot:



<IPython.core.display.Math object>


a_1:



...

<IPython.core.display.Math object>


a_3:



<IPython.core.display.Math object>


a_c3:



<IPython.core.display.Math object>

## Backward recursion

In [ ]:
def NE_backward(R, r, r_c, Ac, Wi_dot, Wi, I, q, m):
  n = len(q)
  f_nsucc = np.zeros((n,1))
  t_nsucc = np.zeros((n,1))
  z0 = np.array([
      [0],
      [0],
      [1]
  ])
  for i in range(n-1, 0, -1):
    Ri = R[i]
    Aci = Ac[i]
    mi = m[i]
    fi = np.dot(Ri,f_nsucc) + mi*Aci
    rci = r_c[i]
    ri = r[i]
    Ii = I[i]
    wi = Wi[i]
    wi_dot = Wi_dot[i]
    term1 = np.cross(np.dot(Ri, fi).reshape(1,3), rci.reshape(1,3)).reshape(3,1)
    term2 = np.cross(fi.reshape(1,3), (ri+rci).reshape(1,3)).reshape(3,1)
    term3 = np.cross(wi.reshape(1,3), (Ii*wi).reshape(1,3)).reshape(3,1)
    ti = np.dot(Ri,t_nsucc) + term1 - term2 + Ii*wi_dot + term3
    ui = np.dot(ti.T,z0)

    fi_l = sp.latex(sp.simplify(fi))
    print(f"f_{i+1}:\n")
    display(Math(fi_l))
    ti_l = sp.latex(sp.simplify(ti))
    print(f"t_{i+1}:\n")
    display(Math(ti_l))
    ui_l = sp.latex(sp.simplify(ui))
    print(f"u_{i+1}:\n")
    display(Math(ui_l))

    f_nsucc = fi
    t_nsucc = ti

  return

In [ ]:
dh = create_DH_table([0, 0, 0], [sp.symbols('a1'),sp.symbols('a2'),sp.symbols('a2')],[0,0,0], [sp.symbols('q1'), sp.symbols('q2'),sp.symbols('q3')])
A,R,r = define_all_matrices(3, dh)

m=[sp.symbols('m1'), sp.symbols('m2'), sp.symbols('m3')]
r_c = [np.array([[sp.symbols('d1')], [0], [0]]), np.array([[sp.symbols('d2')], [0], [0]]), np.array([[sp.symbols('d3')], [0], [0]])]

I = [0,0,0]
NE_backward(R, r, r_c, Aci, Wi_dot, Wi, I, q, m)

i = 1
alpha = 0
a = a1
d = 0
theta = q1

i = 2
alpha = 0
a = a2
d = 0
theta = q2

i = 3
alpha = 0
a = a2
d = 0
theta = q3


Matrix A associated to RF1 wrt RF0

[[cos(q1) -sin(q1) 0 a1*cos(q1)]
 [sin(q1) cos(q1) 0 a1*sin(q1)]
 [0 0 1 0]
 [0 0 0 1]]


Matrix A associated to RF2 wrt RF1

[[cos(q2) -sin(q2) 0 a2*cos(q2)]
 [sin(q2) cos(q2) 0 a2*sin(q2)]
 [0 0 1 0]
 [0 0 0 1]]


Matrix A associated to RF3 wrt RF2

[[cos(q3) -sin(q3) 0 a2*cos(q3)]
 [sin(q3) cos(q3) 0 a2*sin(q3)]
 [0 0 1 0]
 [0 0 0 1]]

f_3:



<IPython.core.display.Math object>

t_3:



<IPython.core.display.Math object>

u_3:



...

<IPython.core.display.Math object>

t_2:



<IPython.core.display.Math object>

u_2:



<IPython.core.display.Math object>

# Stability

In [ ]:
def find_equilibrium(equation, variables):

    if not isinstance(equation, Eq):
        equation = Eq(equation, 0)

    equilibrium_points = solve(equation, variables)

    return equilibrium_points

In [ ]:
# define here the function
B = symbols('B')
K = symbols('K')
D = symbols('D')
F = symbols('F')
M = symbols('M')
g = symbols('g')

t = symbols('t')  # Time variable
theta = Function('theta')(t)  # Define theta as a function of time
theta_dot = theta.diff(t)
theta_ddot = theta_dot.diff(t)

q = Function('q')(t)  # Define theta as a function of time
q_dot = q.diff(t)
q_ddot = q_dot.diff(t)

tau = symbols('tau')

equation1 = Eq(B*theta_ddot+K*(theta-q)+D*(theta_dot-q_dot)+F*theta_dot-tau, 0)
equation2 = Eq(M*theta_ddot+K*(q-theta)-M*g+D*(q_dot-theta_dot)+F*q_dot, 0)

display(Math(sp.latex(equation1)))

display(Math(sp.latex(equation2)))

eq_pnt = {q:0, theta:0}

print("\nBy substituting equilibrium points in the first eq:\n")
display(Math(sp.latex(simplify(equation1.subs(eq_pnt)))))

print("\nBy substituting equilibrium points in the second eq:\n")
display(Math(sp.latex(simplify(equation2.subs(eq_pnt)))))


<IPython.core.display.Math object>

<IPython.core.display.Math object>


By substituting equilibrium points in the first eq:



<IPython.core.display.Math object>


By substituting equilibrium points in the second eq:



<IPython.core.display.Math object>

## Lyapunov Candidate

In [ ]:
def Lyapunov_candidate(V, eq_points):
  print("\nLyapunov candidate:\n")
  display(Math(sp.latex(V)))

  print("\nVERIFY ON YOUR OWN IF THE CANDIDATE IS POSITIVE DEFINITE...\n")

  pt = simplify(V.subs(eq_points))
  print("\nBy substituting in V the equilibrium points we get:\n")
  display(Math(sp.latex(pt)))

  # Compute the partial derivatives
  dV_dtheta = diff(V, theta)
  dV_dq = diff(V, q)
  dV_dtheta_dot = diff(V, theta_dot)
  dV_dq_dot = diff(V, q_dot)

  V_dot = dV_dtheta * theta_dot + dV_dq * q_dot + dV_dtheta_dot * theta_ddot + dV_dq_dot * q_ddot
  print("\nLyapunov derivative:\n")
  display(Math(sp.latex(sp.simplify(V_dot))))

  print("\nVERIFY ON YOUR OWN IF THE DERIVATIVE IS <= 0...\n")
  y_n = input("Is V\dot <= 0? (Y/N): ")
  if y_n == 'Y':
    print("Sufficient condition of stability is satisfied...")
    print(f"\nLaSalle (V\dot = 0 iff q\dot = theta\dot = 0)...\nV\dot = {simplify(V_dot.subs(values))}")
    if simplify(V_dot.subs(values)) == 0:
      print("\nSufficient condition of asymptotic stability is satisfied!\n")

  if y_n == 'N':
    print("Sufficient condition of stability is NOT satisfied...Proceed regulating the system with a control law!")

  return




In [ ]:
B, K, Kp, D, F, M, g = symbols('B K Kp D F M g', real=True)

t = symbols('t')  # Time variable
theta = Function('theta')(t)  # Define theta as a function of time
theta_d = Function('theta_d')(t)  # Define theta as a function of time

theta_dot = theta.diff(t)
theta_ddot = theta_dot.diff(t)

q = Function('q')(t)  # Define theta as a function of time
q_d = Function('q_d')(t)  # Define theta as a function of time

q_dot = q.diff(t)
q_ddot = q_dot.diff(t)

tau = symbols('tau')
V = 0.5*B*theta_dot**2 + 0.5*M*q_dot**2 + 0.5*K*(theta-q)**2 - M*g*q + 0.5*Kp*(theta_d - theta)**2 - M*g*(theta_d-theta)
values = {theta: theta_d, q:q_d, theta_dot:0, q_dot:0}

In [ ]:
Lyapunov_candidate(V, values)


Lyapunov candidate:



<IPython.core.display.Math object>


VERIFY ON YOUR OWN IF THE CANDIDATE IS POSITIVE DEFINITE...


By substituting in V the equilibrium points we get:



<IPython.core.display.Math object>


Lyapunov derivative:



<IPython.core.display.Math object>


VERIFY ON YOUR OWN IF THE DERIVATIVE IS <= 0...

Is V\dot <= 0? (Y/N): Y
Sufficient condition of stability is satisfied...

LaSalle (V\dot = 0 iff q\dot = theta\dot = 0)...
V\dot = 0

Sufficient condition of asymptotic stability is satisfied...

